In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
fish_data=pd.read_csv('/content/drive/MyDrive/FishStatPj/kmni/fish_landings.csv')
weather_data=pd.read_csv('/content/drive/MyDrive/FishStatPj/kmni/monthly_weather_data_malaysia.csv')

In [8]:
fish_data.head()

,date,coast,state,landings
0,2018-01-01,all,Malaysia,97405
1,2018-02-01,all,Malaysia,97467
2,2018-03-01,all,Malaysia,117813
3,2018-04-01,all,Malaysia,127202
4,2018-05-01,all,Malaysia,129453


In [9]:
weather_data.head()


,year_month,place,city,state,temperature,pressure,dew_point,humidity,wind_speed,gust,wind_chill,uv_index,feels_like_temperature,visibility,pollutant_value
0,1996-08,Batu Maung,Bayan Lepas,Pulau Pinang,25.916667,1009.431667,23.652174,87.434783,7.750000,NaN,25.545455,0.0,27.652174,8.560000,NaN
1,1996-08,Kota Sentosa,Kuching,Sarawak,26.086957,1005.990435,23.260870,85.913043,5.266667,NaN,26.086957,0.0,27.652174,8.913043,NaN
2,1996-08,Sepang,Sepang,Kuala Lumpur,25.900000,1007.737727,23.350000,86.500000,8.692308,NaN,25.333333,0.0,27.100000,6.800000,NaN
3,1996-08,Tanjung Aru,Kota Kinabalu,Sabah,26.608696,1007.690000,24.086957,86.565217,6.454545,NaN,26.608696,0.0,28.782609,9.000000,NaN
4,1996-09,Batu Maung,Bayan Lepas,Pulau Pinang,25.714286,1006.782857,24.285714,91.857143,3.000000,NaN,25.714286,0.0,27.142857,9.000000,NaN


## Data Ingestion, Initial Exploration, and Preprocessing

This section details the initial steps taken to prepare the raw fish landings and weather data for subsequent predictive modeling. This phase encompasses data ingestion, initial exploratory data analysis to understand data characteristics, and comprehensive preprocessing steps to clean, standardize, and integrate the two disparate datasets into a unified and usable format.

### 1. Data Overview Before Preprocessing

**A. Fish Landings Data (`fish_data`)**

*   **Source**: Raw fish landings data was obtained from `fish_landings.csv`.
*   **Key Columns**: Initially, the dataset comprised columns such as `date`, `coast`, `state`, and `landings`.
*   **Date Format**: The `date` column was present as a string in 'YYYY-MM-DD' format, requiring conversion to datetime objects for time-series analysis.
*   **States**: The `state` column contained both specific Malaysian states (e.g., 'Perlis', 'Kedah', 'Johor') and aggregated entries like 'Malaysia' and 'All States'. These aggregated entries needed to be handled to focus on state-specific analyses.
*   **Timeframe**: The `fish_data` covered monthly landings from January 2018 to December 2023.

**B. Monthly Weather Data (`weather_data`)**

*   **Source**: Monthly weather metrics were sourced from `monthly_weather_data_malaysia.csv`.
*   **Key Columns**: This dataset included `year_month`, `place`, `city`, `state`, and a range of environmental variables such as `temperature`, `pressure`, `dew_point`, `humidity`, `wind_speed`, `gust`, `wind_chill`, `uv_index`, `feels_like_temperature`, `visibility`, and `pollutant_value`.
*   **State Naming Inconsistencies**: Initial exploration revealed minor inconsistencies in state naming; specifically, 'Labuan' in the weather data needed to be mapped to 'W.P. Labuan' to align with the fish landings data.
*   **Missing Values**: Several weather-related columns were observed to contain missing values, necessitating imputation strategies.
*   **Timeframe**: The `weather_data` spanned from August 1996 onwards, providing ample overlap with the fish landings data for correlation analysis.

### 2. Data Preprocessing Steps

The following sequential steps were performed to clean, standardize, and integrate the `fish_data` and `weather_data`:

1.  **Date Standardization and Feature Creation in `fish_data`**: The `date` column in `fish_data` was converted to `datetime` objects. A new column, `year_month`, was then extracted in 'YYYY-MM' string format from the `date` column. This `year_month` column served as a consistent temporal key for monthly aggregation and subsequent merging.

2.  **State Harmonization and Filtering**: To ensure consistency across datasets, a state mapping was applied to `weather_data`, converting 'Labuan' to 'W.P. Labuan'. Both `fish_data` and `weather_data` were then filtered to include only specific Malaysian states, excluding aggregated entries like 'Malaysia' and 'All States' from `fish_data` that did not represent individual state data.

3.  **Monthly Aggregation of Datasets**:
    *   The filtered `weather_data` was grouped by `state` and `year_month`, and the mean of all numerical weather features was calculated. This provided average monthly weather conditions for each state.
    *   Concurrently, the filtered `fish_data` was grouped by `state` and `year_month`, and the `landings` were summed to obtain total monthly fish landings per state.

4.  **Dataset Merging**: The aggregated `fish_data` (now `fish_agg`) and `weather_data` (now `weather_agg`) were merged into a new DataFrame, `final_df`. This merge was performed using a `left` join on the `state` and `year_month` columns, ensuring that all monthly fish landings entries were retained and matched with their corresponding monthly average weather data.

5.  **Missing Value Imputation**: Missing values (NaNs) in the weather-related columns within `final_df` were addressed through a two-step imputation strategy:
    *   **Linear Interpolation**: For all weather-related numerical columns (e.g., `temperature`, `pressure`, `gust`, etc.), linear interpolation was applied. This method filled missing values by estimating them based on the values of adjacent data points, performed independently for each `state` to account for regional variations.
    *   **State-wise Mean Imputation**: Following linear interpolation, any remaining NaNs (which could occur at the beginning or end of a state's time series) were replaced with the mean value of their respective column for that specific `state`. This ensured complete data for all states.
    *   **Overall Mean Imputation for Visibility**: Any residual missing values in the `visibility` column, after the state-wise imputation, were filled with the overall mean of the `visibility` column across the entire `final_df`. The `final_df.info()` output confirmed the successful imputation of all numerical columns, yielding a complete dataset.

### 3. Data Overview After Preprocessing

Upon completion of the preprocessing steps, the `final_df` emerged as a clean, integrated, and standardized dataset ready for further analysis and modeling. The key characteristics of the processed data are:

*   **Dimensions**: The `final_df` consists of 1008 rows and 14 columns, representing monthly records for various Malaysian states.
*   **Columns**: The dataset now includes `state`, `year_month`, `landings` (total monthly landings), and 10 weather-related features (`temperature`, `pressure`, `dew_point`, `humidity`, `wind_speed`, `gust`, `wind_chill`, `uv_index`, `feels_like_temperature`, `visibility`, `pollutant_value`).
*   **Completeness**: All numerical columns have been successfully imputed, ensuring no missing values remain. This was verified by `final_df.info()` which showed 1008 non-null counts for all columns.
*   **Consistency**: The `year_month` column is uniformly formatted as 'YYYY-MM' strings, providing a consistent time index for time-series analysis.
*   **Aggregation Level**: Data is aggregated at a monthly, state-specific level, allowing for direct investigation into the relationship between monthly weather conditions and fish landings in each state.

This prepared dataset is now suitable for exploratory data analysis (EDA) and subsequent predictive modeling, which will be detailed in the following sections or notebooks.

In [10]:
fish_data['date'].unique()

array(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
       '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
       '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
       '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
       '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
       '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
       '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
       '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
       '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
       '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
       '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
       '2023-01-01', '2023-02-01', '2023-03-01', '2023-

In [11]:
fish_data['state'].unique()

array(['Malaysia', 'All States', 'Perlis', 'Kedah', 'Pulau Pinang',
       'Perak', 'Selangor', 'Negeri Sembilan', 'Melaka', 'Johor',
       'Kelantan', 'Terengganu', 'Pahang', 'Sarawak', 'Sabah',
       'W.P. Labuan'], dtype=object)

In [12]:
weather_data['state'].unique()

array(['Pulau Pinang', 'Sarawak', 'Kuala Lumpur', 'Sabah', 'Selangor',
       'Johor', 'Labuan', 'Kedah', 'Melaka', 'Terengganu', 'Pahang',
       'Kelantan', 'Perak', 'Negeri Sembilan', 'Putrajaya', 'Perlis'],
      dtype=object)

In [13]:
import pandas as pd

fish_data['date'] = pd.to_datetime(fish_data['date'])
fish_data['year_month'] = fish_data['date'].dt.to_period('M').astype(str)

In [14]:
state_map = {
    'Labuan': 'W.P. Labuan'
}
weather_data['state'] = weather_data['state'].replace(state_map)

valid_states = [s for s in fish_data['state'].unique() if s not in {'Malaysia', 'All States'}]

# Filter weather data
weather_filtered = weather_data[weather_data['state'].isin(valid_states)].copy()

In [15]:
weather_agg = (
    weather_filtered
    .groupby(['state', 'year_month'], as_index=False)
    .mean(numeric_only=True)
)

In [16]:
fish_filtered = fish_data[
    fish_data['state'].isin(valid_states)
].copy()

fish_agg = (
    fish_filtered
    .groupby(['state', 'year_month'], as_index=False)
    .sum(numeric_only=True)  # sum landings
)

In [17]:
final_df = fish_agg.merge(
    weather_agg,
    on=['state', 'year_month'],
    how='left'
)

In [18]:
final_df

,state,year_month,landings,temperature,pressure,dew_point,humidity,wind_speed,gust,wind_chill,uv_index,feels_like_temperature,visibility,pollutant_value
0,Johor,2018-01,3562,26.902778,1002.763111,24.155556,85.529167,8.451389,NaN,26.902778,1.887500,29.644789,8.662963,17.793651
1,Johor,2018-02,3828,26.409402,1006.862917,22.599302,80.225594,12.489856,40.020000,26.409952,1.941362,28.354439,8.427652,26.615754
2,Johor,2018-03,5367,27.211806,1004.536944,24.157986,84.222222,7.836806,NaN,27.211806,2.072917,30.106617,8.553830,22.940183
3,Johor,2018-04,5405,27.542361,1003.660222,24.669444,85.149306,7.387500,40.416667,27.542361,1.984722,30.866331,8.518706,24.961115
4,Johor,2018-05,6312,27.450941,1003.817312,24.942204,87.022849,6.360887,33.000000,27.450941,1.707661,30.805753,8.273917,24.824511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,W.P. Labuan,2023-08,1848,27.669355,1006.416747,23.913978,80.778226,3.455645,35.000000,27.669355,2.916667,30.737903,8.950067,47.275538
1004,W.P. Labuan,2023-09,1817,27.400000,1005.953478,24.175362,83.268116,3.526087,35.500000,27.400000,3.030435,30.504348,8.600871,48.293056
1005,W.P. Labuan,2023-10,1667,28.194892,1007.030161,25.021505,83.485215,3.385753,34.333333,28.194892,2.936828,32.489247,8.825269,50.498656
1006,W.P. Labuan,2023-11,1405,28.155172,1005.887256,24.887931,83.002874,3.834770,31.000000,28.155172,2.548851,32.277299,8.856115,29.802450


In [19]:
final_df.isnull().sum()

,0
state,0
year_month,0
landings,0
temperature,98
pressure,98
dew_point,98
humidity,98
wind_speed,98
gust,187
wind_chill,98


In [20]:
cols = ['temperature','pressure','dew_point','humidity','wind_speed','gust',
        'wind_chill','uv_index','feels_like_temperature','visibility','pollutant_value']

# Sort by state and year_month first
final_df = final_df.sort_values(['state','year_month'])

# Apply linear interpolation per state
for col in cols:
    final_df[col] = final_df.groupby('state')[col].transform(lambda x: x.interpolate(method='linear'))
for col in cols:
    final_df[col] = final_df.groupby('state')[col].transform(lambda x: x.interpolate(method='linear').fillna(x.mean()))

In [21]:
overall_mean = final_df['visibility'].mean(skipna=True)
final_df['visibility'] = final_df['visibility'].fillna(overall_mean)

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   1008 non-null   object 
 1   year_month              1008 non-null   object 
 2   landings                1008 non-null   int64  
 3   temperature             1008 non-null   float64
 4   pressure                1008 non-null   float64
 5   dew_point               1008 non-null   float64
 6   humidity                1008 non-null   float64
 7   wind_speed              1008 non-null   float64
 8   gust                    1008 non-null   float64
 9   wind_chill              1008 non-null   float64
 10  uv_index                1008 non-null   float64
 11  feels_like_temperature  864 non-null    float64
 12  visibility              1008 non-null   float64
 13  pollutant_value         1008 non-null   float64
dtypes: float64(11), int64(1), object(2)
memo

In [23]:
final_df.to_csv('/content/drive/MyDrive/FishStatPj/kmni/processed_data.csv',index=False)

In [24]:
final_df.head(5)

,state,year_month,landings,temperature,pressure,dew_point,humidity,wind_speed,gust,wind_chill,uv_index,feels_like_temperature,visibility,pollutant_value
0,Johor,2018-01,3562,26.902778,1002.763111,24.155556,85.529167,8.451389,29.620078,26.902778,1.887500,29.644789,8.662963,17.793651
1,Johor,2018-02,3828,26.409402,1006.862917,22.599302,80.225594,12.489856,40.020000,26.409952,1.941362,28.354439,8.427652,26.615754
2,Johor,2018-03,5367,27.211806,1004.536944,24.157986,84.222222,7.836806,40.218333,27.211806,2.072917,30.106617,8.553830,22.940183
3,Johor,2018-04,5405,27.542361,1003.660222,24.669444,85.149306,7.387500,40.416667,27.542361,1.984722,30.866331,8.518706,24.961115
4,Johor,2018-05,6312,27.450941,1003.817312,24.942204,87.022849,6.360887,33.000000,27.450941,1.707661,30.805753,8.273917,24.824511
